<a href="https://colab.research.google.com/github/SanGyuk-Raccoon/python/blob/master/Mnist_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from torchvision import transforms, datasets

In [4]:
lr = 1e-3
batch_size = 64
num_epoch = 10

ckpt_dir = './checkpoint'
log_dir = './log'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [18]:
class Net(nn.Module) :
  def __init__(self) :
    super(Net, self).__init__()

    self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 10, kernel_size = 5, stride = 1, padding = 0, bias = True)
    self.pool1 = nn.MaxPool2d(kernel_size = 2)
    self.relu1 = nn.ReLU()

    self.conv2 = nn.Conv2d(in_channels = 10, out_channels = 20, kernel_size = 5, stride = 1, padding = 0, bias = True)
    self.drop2 = nn.Dropout2d(p = 0.5)
    self.pool2 = nn.MaxPool2d(kernel_size = 2)
    self.relu2 = nn.ReLU()

    self.fc1 = nn.Linear(in_features = 320, out_features = 50, bias = True)
    self.relu1_fc1 = nn.ReLU()
    self.drop1_fc1 = nn.Dropout2d(p = 0.5)

    self.fc2 = nn.Linear(in_features = 50, out_features = 10, bias = True)

  def forward(self, x) :
    x = self.conv1(x)
    x = self.pool1(x)
    x = self.relu1(x)

    x = self.conv2(x)
    x = self.drop2(x)
    x = self.pool2(x)
    x = self.relu2(x)

    x = x.view(-1, 320)

    x = self.fc1(x)
    x = self.relu1_fc1(x)
    x = self.drop1_fc1(x)

    x = self.fc2(x)

    return x

def save(ckpt_dir, net, optim, epoch) :
  if not os.path.exists(ckpt_dir) :
    os.makedirs(ckpt_dir)

  torch.save({'net': net.state_dict(), 'optim' : optim.state_dict()},
             f'./{ckpt_dir}/model_epoch{epoch}')
  
def load(ckpt_dir, net, optim) :
  ckpt_lst = os.listdir(ckpt_dir)
  ckpt_lst.sort()

  dict_model = torch.load(f'./{ckpt_dir}/{ckpt_lst[-1]}')

  net.load_state_dict(dict_model['net'])
  optim.load_state_dict(dict_model['optim'])

  return net, optim

In [ ]:
transform = transforms.Compose( [transforms.ToTensor(), transforms.Normalize(mean = (0.5, ), std = (0.5, ))] )

dataset = datasets.MNIST(download = True, root = './', train = True, transform = transform)
loader = DataLoader(dataset, batch_size = batch_size, shuffle = True, num_workers = 0)

num_data = len(loader.dataset)
num_batch = np.ceil(num_data / batch_size)

In [19]:
net = Net().to(device)
params = net.parameters()

fn_loss = nn.CrossEntropyLoss().to(device)
fn_pred = lambda output : torch.softmax(output, dim = 1)
fn_acc = lambda pred, label : ( (pred.max(dim = 1)[1] == label).type(torch.float) ).mean()

optim = torch.optim.Adam(params, lr = lr)

writer = SummaryWriter(log_dir = log_dir)

for epoch in range(1, num_epoch + 1) :
  net.train()

  loss_arr = []
  acc_arr = []

  for batch, (input, label) in enumerate(loader, 1) :
    input = input.to(device)
    label = label.to(device)

    output = net(input)
    pred = fn_pred(output)

    optim.zero_grad()

    loss = fn_loss(output, label)
    acc = fn_acc(pred, label)

    loss.backward()

    optim.step()

    loss_arr += [loss.item()]
    acc_arr += [acc.item()]

    
  print(f'TRAIN: {epoch}/{num_epoch} | LOSS: {np.mean(loss_arr) : .4f} | ACC: {np.mean(acc_arr) : .4f}')
  writer.add_scalar('loss', np.mean(loss_arr), epoch)
  writer.add_scalar('acc', np.mean(acc_arr), epoch)

  save(ckpt_dir = ckpt_dir, net = net, optim = optim, epoch = epoch)

writer.close()


TRAIN: 1/10 | LOSS:  0.5832/ | ACC:  0.8136
TRAIN: 2/10 | LOSS:  0.2647/ | ACC:  0.9217
TRAIN: 3/10 | LOSS:  0.2165/ | ACC:  0.9367
TRAIN: 4/10 | LOSS:  0.1919/ | ACC:  0.9429
TRAIN: 5/10 | LOSS:  0.1795/ | ACC:  0.9477
TRAIN: 6/10 | LOSS:  0.1658/ | ACC:  0.9503
TRAIN: 7/10 | LOSS:  0.1615/ | ACC:  0.9527
TRAIN: 8/10 | LOSS:  0.1553/ | ACC:  0.9545
TRAIN: 9/10 | LOSS:  0.1489/ | ACC:  0.9557
TRAIN: 10/10 | LOSS:  0.1429/ | ACC:  0.9575


In [24]:
dataset = datasets.MNIST(download = True, root = './', train = False, transform = transform)
loader = DataLoader(dataset, batch_size = batch_size, shuffle = False, num_workers = 0)

num_data = len(loader.dataset)
num_batch = np.ceil(num_data / batch_size)

net, optim = load(ckpt_dir = ckpt_dir, net = net, optim = optim)

with torch.no_grad() :
  net.eval()

  loss_arr = []
  acc_arr = []

  for batch, (input, label) in enumerate(loader, 1) :
    input = input.to(device)
    label = label.to(device)

    output = net(input)
    pred = fn_pred(output)

    loss = fn_loss(output, label)
    acc = fn_acc(pred, label)

    loss_arr += [loss.item()]
    acc_arr += [acc.item()]

    print(f'TEST| BATCH {batch : 4.0f}/{num_batch : 4.0f} | LOSS: {np.mean(loss_arr) : .4f} | ACC: {np.mean(acc_arr) : .4f}')


TEST| BATCH    1/ 157 | LOSS:  0.0053 | ACC:  1.0000
TEST| BATCH    2/ 157 | LOSS:  0.0059 | ACC:  1.0000
TEST| BATCH    3/ 157 | LOSS:  0.0103 | ACC:  0.9948
TEST| BATCH    4/ 157 | LOSS:  0.0137 | ACC:  0.9922
TEST| BATCH    5/ 157 | LOSS:  0.0186 | ACC:  0.9906
TEST| BATCH    6/ 157 | LOSS:  0.0287 | ACC:  0.9870
TEST| BATCH    7/ 157 | LOSS:  0.0351 | ACC:  0.9866
TEST| BATCH    8/ 157 | LOSS:  0.0417 | ACC:  0.9863
TEST| BATCH    9/ 157 | LOSS:  0.0383 | ACC:  0.9878
TEST| BATCH   10/ 157 | LOSS:  0.0431 | ACC:  0.9859
TEST| BATCH   11/ 157 | LOSS:  0.0422 | ACC:  0.9858
TEST| BATCH   12/ 157 | LOSS:  0.0449 | ACC:  0.9857
TEST| BATCH   13/ 157 | LOSS:  0.0426 | ACC:  0.9856
TEST| BATCH   14/ 157 | LOSS:  0.0421 | ACC:  0.9855
TEST| BATCH   15/ 157 | LOSS:  0.0455 | ACC:  0.9833
TEST| BATCH   16/ 157 | LOSS:  0.0478 | ACC:  0.9824
TEST| BATCH   17/ 157 | LOSS:  0.0488 | ACC:  0.9816
TEST| BATCH   18/ 157 | LOSS:  0.0479 | ACC:  0.9826
TEST| BATCH   19/ 157 | LOSS:  0.0473 | ACC:  

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu1): ReLU()
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (drop2): Dropout2d(p=0.5, inplace=False)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu2): ReLU()
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (relu1_fc1): ReLU()
  (drop1_fc1): Dropout2d(p=0.5, inplace=False)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

'cpu'